In [1]:
import discord
import requests
from bs4 import BeautifulSoup
import nest_asyncio

In [3]:
nest_asyncio.apply()

In [7]:
with open('../keys.json') as fi:
    credentials = json.load(fi)

In [ ]:
token = credentials['api_key']
channel_id = 1336126743223996521

intents = discord.Intents.default()
intents.message_content = True

bot = discord.Client(intents=intents)

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')  
    print("He is risen") 

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')
    for guild in bot.guilds:
        print(f'Connected to: {guild.name} (ID: {guild.id})')
        for channel in guild.text_channels:
            print(f' - {channel.name} (ID: {channel.id})')

@bot.event
async def on_message(message):
    print(f"Received message: {message.content}") 

    if message.author == bot.user:
        return  

    if message.channel.id != channel_id:
        return  

    if message.content.startswith("!rider"):
        print(f"Processing rider query: {message.content}") 
        rider_name = message.content[len("!rider "):].replace(" ", "-").lower()
        url = f"https://www.procyclingstats.com/rider/{rider_name}"

        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        try:
            name_tag = soup.find("h1")
            if name_tag:
                name = name_tag.text.strip()
                await message.channel.send(f"Rider found: **{name}**. More details: {url}")
            else:
                await message.channel.send("Could not find rider")
        except Exception as e:
            print(f"Error occured: {e}")
            await message.channel.send("Error occured")

bot.run(token)